In [1]:
%pip install -q pystac_client

Note: you may need to restart the kernel to use updated packages.


In [ ]:
%pip install git+https://github.com/stac-utils/xpystac

In [ ]:
!mamba install -c conda-forge h5py h5netcdf netcdf4 rioxarray -y

In [ ]:
%pip install pyhdf

In [14]:
!mamba install -c conda-forge gdal -y


Looking for: ['gdal']

conda-forge/linux-64                                        Using cache
conda-forge/noarch                                          Using cache

Pinned packages:
  - python 3.10.*


Transaction

  Prefix: /opt/conda

  All requested packages already installed



In [14]:
!pip uninstall rasterio -y
!mamba install -c conda-forge rasterio -y


Looking for: ['rasterio']

[+] 0.0s
conda-forge/linux-64 ━━━━━━━━━╸━━━━━━━━━━━━━   0.0 B /  ??.?MB @  ??.?MB/s  0.0s[+] 0.1s
conda-forge/linux-64 ━━━━━━━━━━╸━━━━━━━━━━━━   0.0 B /  ??.?MB @  ??.?MB/s  0.1s
conda-forge/noarch   ━━━━━━━━━━━━╸━━━━━━━━━━   0.0 B /  ??.?MB @  ??.?MB/s  0.1s[+] 0.2s
conda-forge/linux-64 ━━━━━━━━━━━━━━━━━━━━━━━   0.0 B /  31.0MB @  ??.?MB/s  0.2s
conda-forge/noarch   ━━━━━━━━╸━━━━━━━━━━━━━━   0.0 B /  ??.?MB @  ??.?MB/s  0.2s[+] 0.3s
conda-forge/linux-64 ━━━━━━━━━━━━━━━━━━━━━━━ 200.8kB /  31.0MB @ 688.4kB/s  0.3s
conda-forge/noarch   ━━━━━━━━━━━━━━━━━━━━━━━  59.3kB /  12.8MB @ 203.4kB/s  0.3s[+] 0.4s
conda-forge/linux-64 ╸━━━━━━━━━━━━━━━━━━━━━━   1.9MB /  31.0MB @   4.8MB/s  0.4s
conda-forge/noarch   ╸━━━━━━━━━━━━━━━━━━━━━━   1.1MB /  12.8MB @   2.8MB/s  0.4s[+] 0.5s
conda-forge/linux-64 ━╸━━━━━━━━━━━━━━━━━━━━━   3.7MB /  31.0MB @   7.5MB/s  0.5s
conda-forge/noarch   ━━╸━━━━━━━━━━━━━━━━━━━━   2.2MB /  12.8MB @   4.5MB/s  0.5s[+] 0.6s
conda-forge/linux-64 ━━━

---

In [1]:
import math

from rastervision.core.box import Box
from rastervision.core.data import (RasterioCRSTransformer, StatsTransformer,
                                    XarraySource)
from rastervision.core.data.raster_source import XarraySource

from rastervision.core.data import Scene
from rastervision.pytorch_learner import (
    SemanticSegmentationRandomWindowGeoDataset)

import numpy as np
import torch
from shapely.geometry import mapping
from matplotlib import pyplot as plt
import seaborn as sns
sns.reset_defaults()

DEVICE = 'cuda' if torch.cuda.is_available() else 'cpu'

In [2]:
import pystac_client
import stackstac

In [3]:
# Tarim basin
# 40.721508, 87.611254
# 40.574240, 87.821939

bbox = Box(ymin=40.721508, xmin=87.611254, ymax=40.574240, xmax=87.821939)
bbox_geometry = mapping(bbox.to_shapely().oriented_envelope)

In [4]:
URL = 'https://cmr.earthdata.nasa.gov/stac/LPCLOUD/'
catalog = pystac_client.Client.open(URL)

search = catalog.search(
    intersects=bbox_geometry,
    collections=['MYD09GA.v061'],
    datetime='2023-11-25/2023-11-30',
)
items = search.get_all_items()

In [9]:
from rastervision.pipeline.file_system.utils import json_to_file, file_to_json
json_to_file(items.to_dict(), 'modis_items.json')

In [10]:
from pystac import ItemCollection
items = ItemCollection.from_file('modis_items.json')

In [ ]:
%pip install -q 'odc-stac[botocore]'

In [5]:
import odc.stac

In [6]:
odc.stac.load(items, crs='epsg:3857', resolution=500)

Aborting load due to failure while reading: https://data.lpdaac.earthdatacloud.nasa.gov/lp-prod-protected/MYD09GA.061/MYD09GA.A2023329.h24v04.061.2023331025647/MYD09GA.A2023329.h24v04.061.2023331025647.hdf:1


RasterioIOError: '/vsicurl/https://data.lpdaac.earthdatacloud.nasa.gov/lp-prod-protected/MYD09GA.061/MYD09GA.A2023329.h24v04.061.2023331025647/MYD09GA.A2023329.h24v04.061.2023331025647.hdf' not recognized as a supported file format.

In [8]:
crs_transformer = RasterioCRSTransformer(transform=data_array.transform, image_crs=data_array.crs)
bbox_pixel_coords = crs_transformer.map_to_pixel(bbox).normalize()
bbox_pixel_coords

Box(ymin=2179, xmin=1158, ymax=2213, xmax=1179)

In [13]:
items[0].properties

{'created': '2023-11-28T02:43:26Z',
 'updated': '2023-11-27T21:17:16.577000Z',
 'datetime': None,
 'platform': 'aqua',
 'proj:epsg': None,
 'proj:wkt2': 'PROJCS["unnamed",GEOGCS["Unknown datum based upon the custom spheroid",DATUM["Not specified (based on custom spheroid)",SPHEROID["Custom spheroid",6371007.181,0]],PRIMEM["Greenwich",0],UNIT["degree",0.0174532925199433,AUTHORITY["EPSG","9122"]]],PROJECTION["Sinusoidal"],PARAMETER["longitude_of_center",0],PARAMETER["false_easting",0],PARAMETER["false_northing",0],UNIT["Meter",1],AXIS["Easting",EAST],AXIS["Northing",NORTH]]',
 'proj:shape': [2400, 2400],
 'instruments': ['modis'],
 'end_datetime': '2023-11-24T23:59:59Z',
 'modis:tile-id': '51024004',
 'proj:geometry': {'type': 'Polygon',
  'coordinates': [[[7783653.637667, 4447802.078667],
    [7783653.637667, 5559752.598333],
    [6671703.118, 5559752.598333],
    [6671703.118, 4447802.078667],
    [7783653.637667, 4447802.078667]]]},
 'proj:transform': [463.3127165279169,
  0.0,
  6671